# Extract Historical Data From PurpleAir Sensors
We need to extract historical data from PurpleAir sensors in McKinley Park. To account for seasonal variability, we wanted to collect a 3 week summer dataset (1st-20th July 2021) and a 3 week winter dataset (1st - 20th Jan 2021). In order to collect historical data, we first have to use the PurpleAir API to get the Thingspeak API keys and sensor IDs. We then pull the data from Thingspeak. Unfortunately, Thingspeak has a date limit and does not keep data older than about 3 1/2 months. For this reason, we will pull the oldest and newest data possible.

In [1]:
import os
os.chdir('../')

from datetime import datetime
import pandas as pd
from src.config import PURPLE_AIR_KEY
from src.data import pull_purpleair_data, pull_purpleair_historical

## Create list of Thingspeak IDs and keys

In [2]:
# pull list of PurpleAir sensors from JSON file
sensor_df = pd.read_json("data/pa_sensors.json")

# create dataframe for McKinley Park sensors
pa_sensor_df = pull_purpleair_data(sensor_df, "Chicago", "McKinley Park", PURPLE_AIR_KEY)

In [3]:
pa_sensor_df

,ID,Label,DEVICE_LOCATIONTYPE,THINGSPEAK_PRIMARY_ID,THINGSPEAK_PRIMARY_ID_READ_KEY,THINGSPEAK_SECONDARY_ID,THINGSPEAK_SECONDARY_ID_READ_KEY,Lat,Lon,PM2_5Value,...,pm10_0_atm,isOwner,humidity,temp_f,pressure,AGE,Stats,ParentID,Flag,Voc
0,96035,36th and Paulina,outside,1267967,6AB8IRLJJPKIFME0,1267968,IQGCXB0M8CAHLMS0,41.828648,-87.667904,10.57,...,11.81,0,31,53,1006.6,1,"{""v"":10.57,""v1"":9.21,""v2"":8.01,""v3"":9.1,""v4"":1...",NaN,NaN,NaN
1,96036,36th and Paulina B,NaN,1267969,L3Y8P2IWC9NY4HLO,1267970,GAT5RETWAAN58851,41.828648,-87.667904,12.68,...,12.68,0,NaN,NaN,NaN,1,"{""v"":12.68,""v1"":12.38,""v2"":12.27,""v3"":13.48,""v...",96035.0,1.0,NaN
2,96395,38th & Winchester,outside,1271673,5REK7K675FMHLXCU,1271674,KA997C774A3QL4TS,41.824900,-87.673881,11.24,...,11.97,0,29,53,1005.8,1,"{""v"":11.24,""v1"":10.19,""v2"":9.01,""v3"":10.37,""v4...",NaN,NaN,NaN
3,96396,38th & Winchester B,NaN,1271675,324MUY4V609SF5M0,1271676,1M5DI7CI7619WP4T,41.824900,-87.673881,12.62,...,13.25,0,33,54,1005.3,1,"{""v"":12.62,""v1"":9.94,""v2"":8.83,""v3"":10.34,""v4""...",96395.0,NaN,37.0
4,94975,39th and Damen (NLEI),outside,1261965,UKYER4XQP4R4KLVQ,1261966,Y794X5FD2TG5A0JG,41.822733,-87.675820,24.98,...,28.05,0,20,62,1005.47,1,"{""v"":24.98,""v1"":13.4,""v2"":11.08,""v3"":12.7,""v4""...",NaN,NaN,NaN
5,94976,39th and Damen (NLEI) B,NaN,1261967,YUSL71LBU2F7DJAJ,1261968,699P8NZTP8ELMOQR,41.822733,-87.675820,28.88,...,32.56,0,NaN,NaN,NaN,1,"{""v"":28.88,""v1"":15.41,""v2"":12.73,""v3"":14.36,""v...",94975.0,NaN,NaN


In [4]:
# create a dictionary of primary Thingspeaks IDs and keys
id_dict = {}
for idx in pa_sensor_df[~pa_sensor_df.Label.str.endswith('B')].index:
    id_dict[pa_sensor_df.loc[idx,'ID']]= {
        'ThingspeakID': pa_sensor_df.loc[idx,'THINGSPEAK_PRIMARY_ID'],
        'ThingspeakKey': pa_sensor_df.loc[idx,'THINGSPEAK_PRIMARY_ID_READ_KEY'],
        'Location': pa_sensor_df.loc[idx,'Label']
    }

In [5]:
id_dict

{96035: {'ThingspeakID': '1267967',
  'ThingspeakKey': '6AB8IRLJJPKIFME0',
  'Location': '36th and Paulina'},
 96395: {'ThingspeakID': '1271673',
  'ThingspeakKey': '5REK7K675FMHLXCU',
  'Location': '38th & Winchester'},
 94975: {'ThingspeakID': '1261965',
  'ThingspeakKey': 'UKYER4XQP4R4KLVQ',
  'Location': '39th and Damen (NLEI)'}}

In [6]:
# create a dictionary of secondary purpleair sensors
id_dict_b = {}
for idx in pa_sensor_df[pa_sensor_df.Label.str.endswith('B')].index:
    id_dict_b[pa_sensor_df.loc[idx,'ID']]= {
        'ThingspeakID_B': pa_sensor_df.loc[idx,'THINGSPEAK_PRIMARY_ID'],
        'ThingspeakKey_B': pa_sensor_df.loc[idx,'THINGSPEAK_PRIMARY_ID_READ_KEY']
    }

In [7]:
id_dict_b

{96036: {'ThingspeakID_B': '1267969', 'ThingspeakKey_B': 'L3Y8P2IWC9NY4HLO'},
 96396: {'ThingspeakID_B': '1271675', 'ThingspeakKey_B': '324MUY4V609SF5M0'},
 94976: {'ThingspeakID_B': '1261967', 'ThingspeakKey_B': 'YUSL71LBU2F7DJAJ'}}

## Pull June PurpleAir Data

In [7]:
# define Thingspeak column labels
PARENT_PRIMARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': 'PM1.0 (CF=1) ug/m3',
    'field2': 'PM2.5 (CF=1) ug/m3',
    'field3': 'PM10.0 (CF=1) ug/m3',
    'field4': 'UptimeMinutes',
    'field5': 'ADC',
    'field6': 'Temperature_F',
    'field7': 'Humidity_%',
    'field8': 'PM2.5 (CF=ATM) ug/m3',
}

PARENT_SECONDARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': '0.3um/dl',
    'field2': '0.5um/dl',
    'field3': '1.0um/dl',
    'field4': '2.5um/dl',
    'field5': '5.0um/dl',
    'field6': '10.0um/dl',
    'field7': 'PM1.0 (CF=ATM) ug/m3',
    'field8': 'PM10 (CF=ATM) ug/m3',
}

CHILD_PRIMARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': 'PM1.0 (CF=1) ug/m3',
    'field2': 'PM2.5 (CF=1) ug/m3',
    'field3': 'PM10.0 (CF=1) ug/m3',
    'field4': 'UptimeMinutes',
    'field5': 'RSSI_dbm',
    'field6': 'Atmospheric Pressure',
    'field7': 'gas_sensor',
    'field8': 'PM2.5 (CF=ATM) ug/m3',
}

CHILD_SECONDARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': '0.3um/dl',
    'field2': '0.5um/dl',
    'field3': '1.0um/dl',
    'field4': '2.5um/dl',
    'field5': '5.0um/dl',
    'field6': '10.0um/dl',
    'field7': 'PM1.0 (CF=ATM) ug/m3',
    'field8': 'PM10 (CF=ATM) ug/m3'
}

In [8]:
# create PurpleAir sensor dataframe
pa_sensor_df = pd.DataFrame(columns=PARENT_PRIMARY_COLS.values())
pa_sensor_df['sensor'] = ''

# iterate through primary sensor data
for sensor in id_dict.keys():
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=id_dict[sensor]['ThingspeakID'],
        key=id_dict[sensor]['ThingspeakKey'],
        col_names=PARENT_PRIMARY_COLS,
        start_date=datetime(2021, 8, 1, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_df = pd.concat([pa_sensor_df, df])

In [9]:
pa_sensor_df.head()

,created_at,entry_id,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,sensor,channel
14601,2021-07-25 00:01:48+00:00,NaN,8.61,14.74,14.81,496.0,-63.0,87.0,65.0,14.74,96035,1267967
14602,2021-07-25 00:03:48+00:00,NaN,8.48,13.27,14.07,498.0,-64.0,87.0,66.0,13.27,96035,1267967
14603,2021-07-25 00:05:48+00:00,NaN,7.77,13.74,14.98,500.0,-63.0,86.0,67.0,13.74,96035,1267967
14604,2021-07-25 00:07:48+00:00,NaN,10.36,18.11,19.79,502.0,-62.0,87.0,67.0,18.11,96035,1267967
14605,2021-07-25 00:09:48+00:00,NaN,10.26,18.68,19.58,504.0,-65.0,87.0,66.0,18.68,96035,1267967


In [10]:
pa_sensor_df.created_at.dt.day.nunique()

28

In [11]:
pa_sensor_df.to_csv('data/summer_2021_pa_parent.csv')

In [12]:
# create PurpleAir sensor B dataframe
pa_sensor_b_df = pd.DataFrame(columns=CHILD_PRIMARY_COLS.values())
pa_sensor_b_df['sensor'] = ''

# iterate through primary sensor data
for sensor in id_dict_b.keys():
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=id_dict_b[sensor]['ThingspeakID_B'],
        key=id_dict_b[sensor]['ThingspeakKey_B'],
        col_names=CHILD_PRIMARY_COLS,
        start_date=datetime(2021, 8, 1, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_b_df = pd.concat([pa_sensor_b_df, df])

In [13]:
pa_sensor_b_df.head()

,created_at,entry_id,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,RSSI_dbm,Atmospheric Pressure,gas_sensor,PM2.5 (CF=ATM) ug/m3,sensor,channel
14601,2021-07-25 00:01:50+00:00,NaN,11.16,15.95,16.02,18240.0,0.0,990.22,NaN,15.95,96036,1267969
14602,2021-07-25 00:03:50+00:00,NaN,9.53,14.09,15.45,18240.0,0.0,990.32,NaN,14.09,96036,1267969
14603,2021-07-25 00:05:50+00:00,NaN,9.91,15.81,17.24,18072.0,0.0,990.31,NaN,15.81,96036,1267969
14604,2021-07-25 00:07:51+00:00,NaN,11.02,17.24,18.46,18240.0,0.0,990.20,NaN,17.24,96036,1267969
14605,2021-07-25 00:09:51+00:00,NaN,11.81,18.02,19.37,18240.0,0.0,990.10,NaN,18.02,96036,1267969


In [14]:
pa_sensor_b_df.created_at.dt.day.nunique()

28

In [15]:
pa_sensor_b_df.to_csv('data/summer_2021_pa_child.csv')

# Pull Autumn 2021 Data

In [16]:
# create PurpleAir sensor dataframe
pa_sensor_df2 = pd.DataFrame(columns=PARENT_PRIMARY_COLS.values())
pa_sensor_df2['sensor'] = ''

# iterate through primary sensor data
for sensor in id_dict.keys():
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=id_dict[sensor]['ThingspeakID'],
        key=id_dict[sensor]['ThingspeakKey'],
        col_names=PARENT_PRIMARY_COLS,
        start_date=datetime(2021, 11, 2, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_df2 = pd.concat([pa_sensor_df2, df])

In [17]:
pa_sensor_df2.head()

,created_at,entry_id,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,sensor,channel
81264,2021-10-26 00:00:39+00:00,NaN,1.23,2.59,2.59,3822.0,-64.0,55.0,57.0,2.59,96035,1267967
81265,2021-10-26 00:02:39+00:00,NaN,0.37,1.03,1.25,3824.0,-63.0,55.0,57.0,1.03,96035,1267967
81266,2021-10-26 00:04:39+00:00,NaN,0.93,1.84,2.11,3826.0,-61.0,55.0,57.0,1.84,96035,1267967
81267,2021-10-26 00:06:39+00:00,NaN,0.98,1.67,2.33,3828.0,-65.0,55.0,57.0,1.67,96035,1267967
81268,2021-10-26 00:08:39+00:00,NaN,0.68,1.33,1.33,3830.0,-62.0,55.0,57.0,1.33,96035,1267967


In [18]:
pa_sensor_df2.created_at.dt.day.nunique()

28

In [19]:
pa_sensor_df2.to_csv('data/autumn_2021_pa_parent.csv')

In [20]:
# create PurpleAir sensor B dataframe
pa_sensor_b_df2 = pd.DataFrame(columns=CHILD_PRIMARY_COLS.values())
pa_sensor_b_df2['sensor'] = ''

# iterate through primary sensor data
for sensor in id_dict_b.keys():
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=id_dict_b[sensor]['ThingspeakID_B'],
        key=id_dict_b[sensor]['ThingspeakKey_B'],
        col_names=CHILD_PRIMARY_COLS,
        start_date=datetime(2021, 11, 2, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_b_df2 = pd.concat([pa_sensor_b_df2, df])

In [21]:
pa_sensor_b_df2.head()

,created_at,entry_id,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,RSSI_dbm,Atmospheric Pressure,gas_sensor,PM2.5 (CF=ATM) ug/m3,sensor,channel
81279,2021-10-26 00:00:42+00:00,NaN,5.49,6.37,8.75,18296.0,0.0,996.51,NaN,6.37,96036,1267969
81280,2021-10-26 00:02:41+00:00,NaN,5.98,6.66,8.52,18296.0,0.0,996.51,NaN,6.66,96036,1267969
81281,2021-10-26 00:04:42+00:00,NaN,5.75,6.69,8.58,18296.0,0.0,996.58,NaN,6.69,96036,1267969
81282,2021-10-26 00:06:41+00:00,NaN,5.41,6.29,8.00,18296.0,0.0,996.55,NaN,6.29,96036,1267969
81283,2021-10-26 00:08:42+00:00,NaN,4.47,5.14,6.86,18464.0,0.0,996.59,NaN,5.14,96036,1267969


In [22]:
pa_sensor_b_df2.to_csv('data/autumn_2021_pa_child.csv')